In [1]:
import plotly.express as px
import pandas as pd

# Data Creation

## Source Data

In [2]:
df = pd.read_csv("./df_2020_2023.csv")

/var/folders/yx/gjm2fjs56vl5kkwrb6xf5d2h0000gn/T/ipykernel_24613/1352109313.py:1: DtypeWarning: Columns (46,180,181,183,184,188,189,190,191,194,195,196,197,198,199,204,205,206,207,214,215,216,217,218,219,220,221,223,225,227,234,235,236,237,238,239,244,245,246,249,250,254,255,256,261,263,264,267,268,269,270,278,284,285,303,333) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./df_2020_2023.csv")


In [3]:
df_ngs = pd.read_csv("./df_ngs_2020_2023.csv")

## QB Data

In [4]:
cols = [
    'desc',
    'season',
    'possession_team',
    'defteam',
    'offense_formation',
    'offense_personnel',
    'defenders_in_box',
    'defense_personnel',
    'number_of_pass_rushers',
    'players_on_play',
    'offense_players',
    'defense_players',
    'n_offense',
    'n_defense',
    'route',
    'defense_man_zone_type',
    'defense_coverage_type',
    'yards_gained',
    'no_huddle',
    'qb_dropback',
    'qb_kneel',
    'qb_spike',
    'qb_scramble',
    'pass_length',
    'pass_location',
    'ep',
    'epa',
    'td_prob',
    'qb_hit',
    'sack',
    'cp',
    'cpoe',
    'qb_epa',
    'xyac_epa',
    'xyac_mean_yardage',
    'xyac_median_yardage',
    'xyac_success',
    'time_to_throw',
    'was_pressure',
    'number_of_pass_rushers',
    'incomplete_pass',
    'passer_player_id',
    'passer_player_name',
    'passing_yards',
    'receiver_player_id',
    'receiver_player_name',
    'receiving_yards',
    'rusher_player_id',
    'rusher_player_name',
    'interception',
]

In [5]:
df_detailed = df.query("play_type == 'pass' and season_type=='REG' and penalty==0")[cols]
df_detailed.head(5)

,desc,season,possession_team,defteam,offense_formation,offense_personnel,defenders_in_box,defense_personnel,number_of_pass_rushers,players_on_play,...,incomplete_pass,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,interception
6,(12:39) (Shotgun) 10-J.Garoppolo pass short mi...,2020,SF,ARI,SHOTGUN,"1 RB, 1 TE, 3 WR",5.0,"1 DL, 5 LB, 5 DB",4.0,46078;38547;45069;46113;38551;41325;45185;4781...,...,0.0,00-0031345,J.Garoppolo,11.0,00-0033288,G.Kittle,11.0,NaN,NaN,0.0
9,(11:50) (Shotgun) 1-K.Murray pass short left t...,2020,ARI,SF,SHOTGUN,"1 RB, 1 TE, 3 WR",5.0,"4 DL, 2 LB, 5 DB",4.0,42389;47785;39973;38779;39965;44815;46166;4778...,...,0.0,00-0035228,K.Murray,3.0,00-0030564,D.Hopkins,3.0,NaN,NaN,0.0
10,"(11:24) (No Huddle, Shotgun) 1-K.Murray pass s...",2020,ARI,SF,SHOTGUN,"1 RB, 2 TE, 2 WR",7.0,"4 DL, 3 LB, 4 DB",4.0,42389;47785;39973;38779;44815;39965;46166;4778...,...,0.0,00-0035228,K.Murray,7.0,00-0022921,L.Fitzgerald,7.0,NaN,NaN,0.0
12,(10:58) (Shotgun) 1-K.Murray pass incomplete s...,2020,ARI,SF,EMPTY,"1 RB, 1 TE, 3 WR",5.0,"4 DL, 2 LB, 5 DB",4.0,46203;42389;47785;39973;38779;39965;44815;4616...,...,1.0,00-0035228,K.Murray,NaN,00-0022921,L.Fitzgerald,NaN,NaN,NaN,0.0
13,"(10:55) (No Huddle, Shotgun) 1-K.Murray pass s...",2020,ARI,SF,EMPTY,"1 RB, 0 TE, 4 WR",4.0,"4 DL, 2 LB, 5 DB",4.0,41249;46203;42389;47785;39973;38779;39965;4481...,...,0.0,00-0035228,K.Murray,10.0,00-0022921,L.Fitzgerald,10.0,NaN,NaN,0.0


In [27]:
total_pass = len(df_detailed)

In [29]:
man_cover = len(df_detailed.query("defense_man_zone_type=='MAN_COVERAGE'"))

In [28]:
zone_cover = len(df_detailed.query("defense_man_zone_type=='ZONE_COVERAGE'"))

In [30]:
man_cover / total_pass, zone_cover / total_pass

(0.27150715071507153, 0.6189348346599366)

In [10]:
df_qb = df_detailed.groupby(['passer_player_id', 'season', 'defense_man_zone_type']).agg({
    'passer_player_name': lambda x: x.mode()[0],
    'yards_gained': ['mean'],
    'ep': ['mean'],
    'epa': ['mean'],
    'was_pressure': ['mean'],
    'time_to_throw': ['mean'],
    'qb_dropback': ['sum'],
    'cpoe': ['mean'],
    'qb_epa': ['mean'],
    'qb_hit': ['mean'],
    'sack': ['mean'],
    'interception': ['mean']
}).reset_index()

In [11]:
df_qb.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_qb.columns]

In [12]:
df_qb.head()

,passer_player_id_,season_,defense_man_zone_type_,passer_player_name_<lambda>,yards_gained_mean,ep_mean,epa_mean,was_pressure_mean,time_to_throw_mean,qb_dropback_sum,cpoe_mean,qb_epa_mean,qb_hit_mean,sack_mean,interception_mean
0,00-0019596,2020,MAN_COVERAGE,T.Brady,7.684211,2.513509,0.421116,0.273684,2.513111,190.0,3.752385,0.421116,0.100000,0.0,0.021053
1,00-0019596,2020,ZONE_COVERAGE,T.Brady,7.665835,1.914749,0.155436,0.261845,2.602394,401.0,2.936195,0.184754,0.079800,0.0,0.019950
2,00-0019596,2021,MAN_COVERAGE,T.Brady,7.537255,2.523009,0.312150,0.231373,2.485314,255.0,-1.281554,0.319770,0.066667,0.0,0.007843
3,00-0019596,2021,ZONE_COVERAGE,T.Brady,7.709046,2.084208,0.282239,0.161369,2.506460,409.0,5.875608,0.297470,0.034230,0.0,0.019560
4,00-0019596,2022,MAN_COVERAGE,T.Brady,5.412060,2.440110,0.122861,0.231156,2.440146,199.0,-2.665592,0.122861,0.085427,0.0,0.010050


In [32]:
df_qb.time_to_throw_mean.median(), df_qb.query("defense_man_zone_type_=='MAN_COVERAGE'").time_to_throw_mean.median(), df_qb.query("defense_man_zone_type_=='ZONE_COVERAGE'").time_to_throw_mean.median()

(2.759955882352941, 2.6986206896551725, 2.823253181029497)

In [33]:
df_qb.was_pressure_mean.median(), df_qb.query("defense_man_zone_type_=='MAN_COVERAGE'").was_pressure_mean.median(), df_qb.query("defense_man_zone_type_=='ZONE_COVERAGE'").was_pressure_mean.median()

(0.2985507246376812, 0.3333333333333333, 0.2770513723448776)

In [34]:
df_qb.epa_mean.median(), df_qb.query("defense_man_zone_type_=='MAN_COVERAGE'").epa_mean.median(), df_qb.query("defense_man_zone_type_=='ZONE_COVERAGE'").epa_mean.median()

(0.0742320706764706, 0.08073352753968253, 0.07038871566169255)

In [36]:
df_qb.cpoe_mean.median(), df_qb.query("defense_man_zone_type_=='MAN_COVERAGE'").cpoe_mean.median(), df_qb.query("defense_man_zone_type_=='ZONE_COVERAGE'").cpoe_mean.median()

(-1.7734697465342864, -5.321380377192983, 0.8113494516129023)

In [39]:
df_qb.query("qb_epa_mean >= 0.08 and defense_man_zone_type_=='MAN_COVERAGE'")

,passer_player_id_,season_,defense_man_zone_type_,passer_player_name_<lambda>,yards_gained_mean,ep_mean,epa_mean,was_pressure_mean,time_to_throw_mean,qb_dropback_sum,cpoe_mean,qb_epa_mean,qb_hit_mean,sack_mean,interception_mean
0,00-0019596,2020,MAN_COVERAGE,T.Brady,7.684211,2.513509,0.421116,0.273684,2.513111,190.0,3.752385,0.421116,0.100000,0.0,0.021053
2,00-0019596,2021,MAN_COVERAGE,T.Brady,7.537255,2.523009,0.312150,0.231373,2.485314,255.0,-1.281554,0.319770,0.066667,0.0,0.007843
4,00-0019596,2022,MAN_COVERAGE,T.Brady,5.412060,2.440110,0.122861,0.231156,2.440146,199.0,-2.665592,0.122861,0.085427,0.0,0.010050
6,00-0020531,2020,MAN_COVERAGE,D.Brees,6.122807,2.569421,0.134225,0.271930,2.623702,114.0,-3.230276,0.134225,0.096491,0.0,0.026316
8,00-0022924,2020,MAN_COVERAGE,B.Roethlisberger,6.079832,2.155064,0.100245,0.239496,2.215920,238.0,-1.469431,0.131472,0.126050,0.0,0.016807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,00-0038122,2022,MAN_COVERAGE,D.Ridder,5.038462,2.796515,0.116300,0.269231,2.663154,26.0,1.051184,0.330106,0.076923,0.0,0.000000
671,00-0038391,2023,MAN_COVERAGE,S.Clifford,37.000000,0.501167,2.143627,0.000000,2.870000,1.0,58.596500,2.143627,0.000000,0.0,0.000000
673,00-0038416,2023,MAN_COVERAGE,T.Bagent,6.115385,2.832586,-0.068364,0.230769,2.704885,26.0,-6.482928,0.158856,0.038462,0.0,0.000000
675,00-0038476,2023,MAN_COVERAGE,T.DeVito,5.464286,2.063391,0.139926,0.178571,2.760232,56.0,-5.254798,0.139926,0.000000,0.0,0.000000


In [17]:
# still a producing qb even though we hang onto the ball too much => oline is good? or qb is so good at the pocket?
df_qb.query("qb_epa_mean >= 0 and cpoe_mean >= -0.81 and time_to_throw_mean > 2.78 and qb_dropback_sum >= 100 and was_pressure_mean <= 0.29")

,passer_player_id_,season_,defense_man_zone_type_,passer_player_name_<lambda>,yards_gained_mean,ep_mean,epa_mean,was_pressure_mean,time_to_throw_mean,qb_dropback_sum,cpoe_mean,qb_epa_mean,qb_hit_mean,sack_mean,interception_mean
29,00-0026143,2020,ZONE_COVERAGE,M.Ryan,7.477327,2.043945,0.204266,0.284010,2.935721,419.0,2.114571,0.218249,0.081146,0.0,0.016706
79,00-0027939,2020,ZONE_COVERAGE,C.Newton,8.354369,1.554468,0.236990,0.266990,2.905927,206.0,1.748240,0.236990,0.053398,0.0,0.024272
118,00-0029263,2020,ZONE_COVERAGE,R.Wilson,8.197368,1.982754,0.313623,0.239474,3.006037,380.0,7.915524,0.333145,0.084211,0.0,0.018421
120,00-0029263,2021,ZONE_COVERAGE,R.Wilson,7.748408,1.663692,0.125717,0.257962,2.810350,314.0,2.475830,0.149845,0.108280,0.0,0.019108
122,00-0029263,2022,ZONE_COVERAGE,R.Wilson,7.231183,1.402868,0.107953,0.239247,2.950960,372.0,0.565345,0.107953,0.102151,0.0,0.016129
183,00-0031280,2021,ZONE_COVERAGE,D.Carr,7.365196,1.962181,0.092335,0.272059,2.811257,408.0,4.250324,0.117931,0.066176,0.0,0.019608
197,00-0031345,2023,ZONE_COVERAGE,J.Garoppolo,8.372727,1.784382,0.162902,0.281818,3.019055,110.0,5.717869,0.162902,0.081818,0.0,0.054545
216,00-0031800,2021,ZONE_COVERAGE,T.Heinicke,6.991150,1.835385,0.091305,0.277286,3.043198,339.0,1.480794,0.113197,0.094395,0.0,0.032448
259,00-0032950,2021,ZONE_COVERAGE,C.Wentz,7.633721,2.027688,0.187611,0.284884,2.832032,344.0,0.058382,0.205062,0.116279,0.0,0.014535
275,00-0033106,2020,ZONE_COVERAGE,J.Goff,7.506527,1.900848,0.191848,0.266319,2.822653,383.0,2.676740,0.220487,0.091384,0.0,0.015666


In [18]:
# poor qb play, but with time to throw and low pressure
df_qb.query("qb_epa_mean < 0 and cpoe_mean < -0.81 and time_to_throw_mean > 2.78 and qb_dropback_sum >= 100 and was_pressure_mean <= 0.29")

,passer_player_id_,season_,defense_man_zone_type_,passer_player_name_<lambda>,yards_gained_mean,ep_mean,epa_mean,was_pressure_mean,time_to_throw_mean,qb_dropback_sum,cpoe_mean,qb_epa_mean,qb_hit_mean,sack_mean,interception_mean
41,00-0026158,2023,ZONE_COVERAGE,J.Flacco,7.095238,1.726113,-0.127437,0.277778,2.970952,126.0,-5.866835,-0.046915,0.031746,0.0,0.031746
421,00-0034855,2021,MAN_COVERAGE,B.Mayfield,4.661654,2.171686,-0.072223,0.187970,2.836241,133.0,-10.234452,-0.072223,0.052632,0.0,0.037594
478,00-0035228,2023,ZONE_COVERAGE,K.Murray,6.341463,1.807851,-0.132874,0.186992,2.877756,123.0,-2.061058,-0.132874,0.032520,0.0,0.032520
634,00-0037013,2021,MAN_COVERAGE,Z.Wilson,4.684211,1.973944,-0.217939,0.263158,2.855947,114.0,-17.033672,-0.217939,0.043860,0.0,0.026316


In [19]:
# good qb, we don't know about oline
df_qb.query("qb_epa_mean >= 0 and cpoe_mean >= -0.81 and time_to_throw_mean <= 2.78 and qb_dropback_sum >= 100 and was_pressure_mean <= 0.29")

,passer_player_id_,season_,defense_man_zone_type_,passer_player_name_<lambda>,yards_gained_mean,ep_mean,epa_mean,was_pressure_mean,time_to_throw_mean,qb_dropback_sum,cpoe_mean,qb_epa_mean,qb_hit_mean,sack_mean,interception_mean
0,00-0019596,2020,MAN_COVERAGE,T.Brady,7.684211,2.513509,0.421116,0.273684,2.513111,190.0,3.752385,0.421116,0.100000,0.0,0.021053
1,00-0019596,2020,ZONE_COVERAGE,T.Brady,7.665835,1.914749,0.155436,0.261845,2.602394,401.0,2.936195,0.184754,0.079800,0.0,0.019950
3,00-0019596,2021,ZONE_COVERAGE,T.Brady,7.709046,2.084208,0.282239,0.161369,2.506460,409.0,5.875608,0.297470,0.034230,0.0,0.019560
5,00-0019596,2022,ZONE_COVERAGE,T.Brady,6.782197,1.839733,0.110125,0.185606,2.451763,528.0,1.540555,0.145940,0.070076,0.0,0.013258
7,00-0020531,2020,ZONE_COVERAGE,D.Brees,8.167910,2.235730,0.292024,0.208955,2.671724,268.0,3.006344,0.315674,0.070896,0.0,0.011194
13,00-0022942,2020,ZONE_COVERAGE,P.Rivers,7.628049,1.979912,0.141834,0.240854,2.621534,328.0,2.121656,0.174209,0.070122,0.0,0.024390
15,00-0023436,2020,ZONE_COVERAGE,A.Smith,7.035714,1.651812,0.127237,0.279762,2.618256,168.0,5.088778,0.153207,0.071429,0.0,0.017857
16,00-0023459,2020,MAN_COVERAGE,A.Rodgers,7.848485,2.762507,0.431783,0.272727,2.725096,198.0,6.566257,0.431783,0.060606,0.0,0.010101
17,00-0023459,2020,ZONE_COVERAGE,A.Rodgers,8.452830,1.893574,0.371052,0.188679,2.724418,318.0,7.120744,0.404083,0.050314,0.0,0.009434
18,00-0023459,2021,MAN_COVERAGE,Aa.Rodgers,7.028902,2.650398,0.314721,0.254335,2.608422,173.0,1.670930,0.314721,0.109827,0.0,0.005780
